# Deep Learning (AI5100)
## Assignment 2: <i>Convolutional Neural Networks</i>

<b>Name</b> - Sayanta Adhikari </br>
<b>Roll No</b> - AI22MTECH12005

#### Problems

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 7)
matplotlib.rcParams['figure.facecolor'] = '#00000000'
np.random.seed(33)

1. Convolution Function

In [2]:
# Activation functions

def sigmoid(X: np.ndarray):
    return 1/(1 + np.exp(-1 * X))

def tanh(X: np.ndarray):
    return (np.exp(2 * X) - 1 ) / (np.exp(2 * X) + 1 )

def relu(X: np.ndarray):
    return X * (X > 0)

def prelu(X: np.ndarray, a: float = 0.02):
    return X * (X > 0) + a * X * (X <= 0)

# Padding function

def padding(img: np.ndarray, pad: int, pad_type: str):
    n, m, c = img.shape
    if pad_type == 'zero':
        img_new = np.zeros([n + 2*pad, m + 2*pad, c])
        img_new[pad: -pad, pad: -pad, :] = img
        return img_new
    if pad_type == 'mirror':
        img_new = np.zeros([n + 2*pad, m + 2*pad, c])
        img_new[pad: -pad, pad: -pad, :] = img
        img_new[pad: -pad, 0: pad, :] = img[:, 0: pad, :]
        img_new[pad: -pad, -pad: , :] = img[:, -pad: , :]
        img_new[0: pad, pad: -pad, :] = img[0: pad, :, :]
        img_new[-pad: , pad: -pad, :] = img[-pad: , :, :]
        return img_new

In [ ]:
# convolution function
def convolution(img: np.ndarray, filter: np.ndarray, stride: int, pad: int, pad_type: str, activation_fn: str):
    # Default values
    ACTIVATION_FN = {'relu': relu, 'sigmoid': sigmoid, 'tanh': tanh, 'prelu': prelu}
    PAD_TYPE = {'zero', 'mirror', 'none'}

    # Parameter Checks
    if activation_fn not in ACTIVATION_FN.keys():
        raise ValueError("convolution: activation_fn must be one of %r ." % ACTIVATION_FN)
    if pad_type not in PAD_TYPE:
        raise ValueError("convolution: pad_type must be one of %r ." % PAD_TYPE)

    n, m, c_i = img.shape ; k1, k2, c_f = filter.shape

    if k1 % 2 == 0 or k2 % 2 == 0:
        raise ValueError("convolution: filter shape dimensions must be odd.")
    if c_i != c_f:
        raise ValueError("convolution: img and filter channel dimension must match.")
    if (n + 2* pad - k1)/stride != (n + 2* pad - k1)//stride:
        raise ValueError("convolution: (img.shape[0] + 2*pad - filter.shape[0])/stride must be an integer.")
    if (m + 2* pad - k2)/stride != (m + 2* pad - k2)//stride:
        raise ValueError("convolution: (img.shape[1] + 2*pad - filter.shape[1])/stride must be an integer.")

    # Applying padding
    if pad_type != 'none':
        img = padding(img, pad, pad_type)
        n = n + 2*pad; m = m + 2*pad

    s1 = n - k1 + stride; s2 = m - k2 + stride

    # Performing convolution
    activation = np.zeros([s1/stride, s2/stride])
    k, l = 0, 0
    for i in range(k1//2, s1, stride):
        for j in range(k2//2, s2, stride):
            temp = img[i - k1//2: i + k1//2 + 1, j - k2//2: j + k2//2 + 1, :] * filter
            activation[k, l] = np.sum(temp)
            k +=1; l += 1
    return ACTIVATION_FN[activation_fn](activation)